In [0]:
import io
import keras
import tensorflow as tf
import os
import numpy as np # used for handling numbers
import pandas as pd # used for handling the dataset
df= pd.read_csv('drive/My Drive/Colab Notebooks/dataset/labeled_outs.txt',delimiter=",",header=None)
#test_df=pd.read_csv('drive/My Drive/Colab Notebooks/dataset/test_outs.txt',delimiter=",",header=None)
# Dataset is now stored in a Pandas Dataframe

Using TensorFlow backend.


In [0]:
file = open('drive/My Drive/Colab Notebooks/dataset/vocab.txt', 'r')
# read all text
vocab = file.readlines()
vocabulary={}
for w in vocab:
  vocabulary[w.strip()]=""
# close the file
file.close()

In [0]:
df.columns=['label','text']
df.text=df.text.astype(str)
df.label=df.label.astype(str)
df=df.sample(frac=1)
docs=df['text'].values
labels=df['label'].values

In [0]:
docs_filtered=[]
for d in docs:
  # split into tokens by white space
  tokens = d.split()
  temp=""
  for t in tokens:
    if t in vocabulary:
      temp=temp+" "+t
  docs_filtered.append(temp)


In [0]:
# integer encode the documents
from keras.preprocessing.text import one_hot
vocab_size = len(vocabulary)+1

encoded_docs = [one_hot(d, vocab_size) for d in docs_filtered]

In [0]:
# pad documents to a max length of 4 words
max_length = 200
padded_docs = keras.preprocessing.sequence.pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(len(padded_docs))

91795


In [0]:
#import sys
#import numpy
#numpy.set_printoptions(threshold=sys.maxsize)
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(labels)
encoded_Y = encoder.transform(labels)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
print(dummy_y.shape)


(91795, 8)


In [0]:
emb_dim=32
emb_model = tf.keras.Sequential([ 
    tf.keras.layers.Input(shape=(max_length,)) ,                           
    tf.keras.layers.Embedding(vocab_size, output_dim=emb_dim),
    tf.keras.layers.LSTM(128,return_sequences=True,dropout=0.3),
    tf.keras.layers.LSTM(64,return_sequences=True,dropout=0.2),
    tf.keras.layers.LSTM(32,return_sequences=True,dropout=0.1),
    tf.keras.layers.LSTM(16),
    tf.keras.layers.Dense(8, activation="softmax")
])

emb_model.compile(optimizer='adam',loss= 'categorical_crossentropy',metrics=['acc'])
print(emb_model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 32)           8000032   
_________________________________________________________________
lstm (LSTM)                  (None, 200, 128)          82432     
_________________________________________________________________
lstm_1 (LSTM)                (None, 200, 64)           49408     
_________________________________________________________________
lstm_2 (LSTM)                (None, 200, 32)           12416     
_________________________________________________________________
lstm_3 (LSTM)                (None, 16)                3136      
_________________________________________________________________
dense (Dense)                (None, 8)                 136       
Total params: 8,147,560
Trainable params: 8,147,560
Non-trainable params: 0
______________________________________________

In [0]:
emb_model.fit(padded_docs, dummy_y,epochs=30,batch_size=256, verbose=1,validation_split=0.1)


Epoch 1/30
323/323 [==============================] - 79s 244ms/step - loss: 1.9380 - acc: 0.2110 - val_loss: 1.9169 - val_acc: 0.2185
Epoch 2/30
323/323 [==============================] - 69s 213ms/step - loss: 1.9274 - acc: 0.2214 - val_loss: 1.7298 - val_acc: 0.2948
Epoch 3/30
323/323 [==============================] - 68s 210ms/step - loss: 1.5986 - acc: 0.3408 - val_loss: 1.3733 - val_acc: 0.4512
Epoch 4/30
323/323 [==============================] - 67s 207ms/step - loss: 1.3054 - acc: 0.4701 - val_loss: 1.2724 - val_acc: 0.4939
Epoch 5/30
323/323 [==============================] - 67s 208ms/step - loss: 1.1010 - acc: 0.5490 - val_loss: 1.0821 - val_acc: 0.5564
Epoch 6/30
323/323 [==============================] - 67s 206ms/step - loss: 0.9174 - acc: 0.6272 - val_loss: 0.9664 - val_acc: 0.6209
Epoch 7/30
323/323 [==============================] - 68s 210ms/step - loss: 0.7512 - acc: 0.6886 - val_loss: 0.8087 - val_acc: 0.6880
Epoch 8/30
323/323 [==============================] - 6

In [0]:
emb_model1 = tf.keras.Model(inputs=emb_model.inputs , outputs=emb_model.layers[3].output)
emb_X = emb_model1.predict(padded_docs)
print(emb_X.shape)
#emb_X=np.reshape(emb_X, (emb_X.shape[0],1,-1))

(91795, 200, 32)


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(emb_X, dummy_y, test_size=0.18)

In [0]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(75271, 200, 32)
(75271, 8)
(16524, 200, 32)
(16524, 8)


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D

In [0]:
model = Sequential()
model.add(Conv1D(filters=128, kernel_size=5,input_shape=(X_train.shape[1],X_train.shape[2]), activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(filters=64, kernel_size=5, activation='relu'))
model.add(Dropout(0.4))
model.add(MaxPooling1D(pool_size=1))
#model.add(LSTM(256,return_sequences=True,dropout=0.5,recurrent_dropout=0.3))
model.add(LSTM(128,return_sequences=True,dropout=0.5,recurrent_dropout=0.3))
model.add(LSTM(64,return_sequences=True,dropout=0.4,recurrent_dropout=0.2))
model.add(LSTM(32,return_sequences=True))
model.add(LSTM(16))
model.add(Dense(8, activation='softmax'))

In [0]:

model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=['acc'])
# summarize the model
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 196, 128)          20608     
_________________________________________________________________
dropout_1 (Dropout)          (None, 196, 128)          0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 196, 128)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 192, 64)           41024     
_________________________________________________________________
dropout_2 (Dropout)          (None, 192, 64)           0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 192, 64)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 192, 128)         

In [0]:
history = model.fit(X_train, y_train, epochs=25,batch_size=256, shuffle=False, verbose=1,validation_split=0.1)
#model.save('saved_model/text_classifier_rnn_word_embedding_epoch_20')

Train on 67743 samples, validate on 7528 samples
Epoch 1/25
67743/67743 [==============================] - 446s 7ms/step - loss: 0.4746 - acc: 0.9111 - val_loss: 0.1709 - val_acc: 0.9681
Epoch 2/25
67743/67743 [==============================] - 446s 7ms/step - loss: 0.1709 - acc: 0.9651 - val_loss: 0.1497 - val_acc: 0.9673
Epoch 3/25
67743/67743 [==============================] - 443s 7ms/step - loss: 0.1439 - acc: 0.9695 - val_loss: 0.1083 - val_acc: 0.9795
Epoch 4/25
67743/67743 [==============================] - 447s 7ms/step - loss: 0.1207 - acc: 0.9745 - val_loss: 0.1006 - val_acc: 0.9807
Epoch 5/25
67743/67743 [==============================] - 445s 7ms/step - loss: 0.1107 - acc: 0.9766 - val_loss: 0.0971 - val_acc: 0.9814
Epoch 6/25
67743/67743 [==============================] - 443s 7ms/step - loss: 0.1048 - acc: 0.9777 - val_loss: 0.0880 - val_acc: 0.9821
Epoch 7/25
67743/67743 [==============================] - 448s 7ms/step - loss: 0.1060 - acc: 0.9767 - val_loss: 0.0802 - v

KeyboardInterrupt: ignored

In [0]:
 #model.load_weights('./checkpoints/text_classifier_rnn_word_embedding')
new_model = tf.keras.models.load_model('saved_model/text_classifier_rnn_word_embedding_epoch_20')

In [0]:
results = model.evaluate(X_test, y_test)
print(results)

16524/16524 [==============================] - 67s 4ms/step
[0.06589443249665372, 0.9866860508918762]


In [0]:
y_pred=model.predict(X_test)

In [0]:
"""
import numpy as np
array1=[[ 0,  1,  2,  3,  4],
       [ 0,  1,  2,  3,  4],
       [5, 6, 7, 1, 0],
       [5, 6, 7, 1, 0]]
array2=[[ 5, 6, 7, 1, 0],
       [ 0,  1,  2,  3,  4],
       [0,  1,  2,  3,  4],
       [5, 6, 7, 1, 0]]
"""
no_cat=len(y_test[0])
conf_mat=np.zeros(no_cat*no_cat).reshape(no_cat,no_cat)
test_size=len(y_test)
print(conf_mat.shape)
for i in range(test_size):
  y_t=y_test[i]
  y_p=y_pred[i]
  class_t=np.where(y_t == np.amax(y_t))
  class_t=class_t[0][0]
  class_p=np.where(y_p == np.amax(y_p))
  class_p=class_p[0][0]
  conf_mat[class_t][class_p]=conf_mat[class_t][class_p]+1

(8, 8)


In [0]:
tp=np.zeros(no_cat)
fp=np.zeros(no_cat)
tn=np.zeros(no_cat)
fn=np.zeros(no_cat)

prec=np.zeros(no_cat)
rec=np.zeros(no_cat)
f_score=np.zeros(no_cat)

for i in range(no_cat):
  for j in range(no_cat):
    if i==j:
      tp[i]=conf_mat[i][j]
    if i!= j:
      fp[i] = fp[i]+conf_mat[j][i]
      fn[i] = fn[i]+conf_mat[i][j]
    for k in range(no_cat):
      if i!=k & j!=k:
        tn[i]=tn[i]+conf_mat[j][k]

for i in range(no_cat):
    prec[i] = tp[i] * 100.0 / (tp[i] + fp[i])
    rec[i] = tp[i] * 100.0 / (tp[i] + fn[i])
    f_score[i] = 2 * prec[i] * rec[i] / (prec[i] + rec[i])

print(np.average(prec))
print(np.average(rec))
print(np.average(f_score))

98.69961780068988
98.61391626039114
98.6559211899042
